In [9]:
import random
import os, sys

import numpy as np
import torch
import pandas as pd
from torch.utils.data import Subset
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import StratifiedKFold

from dataset import MaskBaseDataset, MaskSplitByProfileDataset, TestDataset, CustomAugmentation
import model 
from loss import FocalLoss
from efficientnet_pytorch import EfficientNet

import time
import datetime

SEED = 123

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
# 생성된 모델을 저장할 폴더 (mask, gender, age)
# for 문에서 사용할 변수들
RESULT_PATH = ['k-fold-mask-b4', 'k-fold-gender-b4', 'k-fold-age-b4']
TARGET = ['mask','gender', 'age']
CLASS = [3, 2, 3]
# if not os.path.exists(RESULT_PATH):
#     os.mkdir(RESULT_PATH)

In [3]:
epochs = 10
batch_size = 64
num_workers = 4
learning_rate = 1e-4
lr_decay_step = 5
classes_num = 18
criterion_name = 'focal'

train_log_interval = 20
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# DataLoader 생성 함수 (baseline과 동일)
def get_dataloader(dataset, train_idx, valid_idx, batch_size, num_workers):
    train_set = torch.utils.data.Subset(dataset, indices=train_idx)
    val_set = torch.utils.data.Subset(dataset, indices=valid_idx)
    
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last = True,
        shuffle=True
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=False
    )
    
    return train_loader, val_loader


In [5]:
# Dataset 로드 클래스 (baseline에서 class_by 부분 추가=> __init__, __getitem__)
dataset = MaskSplitByProfileDataset(data_dir = '../../input/data/train/images', class_by='mask')
transform = CustomAugmentation(resize=(256, 256), mean=dataset.mean, std=dataset.std)
dataset.set_transform(transform)

In [6]:
dataset.__len__()

6634

In [7]:
def efficientnetb4(classes):
    model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=classes)

    return model

In [8]:
def undersampling(k, inputs, labels):
    if k == 0:
        if labels == 0:
            return np.random.randint(3)
        else:
            return 0
    elif k == 1:
        return 0
    else:
        if labels != 2:
            return np.random.randint(3)
        else:
            return 0

In [10]:
# for 문을 3번 (mask, gender, age)을 반복 합니다.
for k in range(3):
    
    # 각 k-fold 별로 생성되는 모델을 저장할 폴더 생성
    if not os.path.exists(RESULT_PATH[k]):
        os.mkdir(RESULT_PATH[k])
    
    # 각 모델별로 dataset 생성
    # ex) class_by = TARGET[0] -> class_by='mask'
    dataset = MaskSplitByProfileDataset(data_dir = '../../input/data/train/images', class_by=TARGET[k])
    transform = CustomAugmentation(resize=(256, 256), mean=dataset.mean, std=dataset.std)
    dataset.set_transform(transform)
    
    # k-fold 적용을 위한 k 값 설정
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits)
    
    
    # patience = 2 -> 2번동안 더 좋은 모델이 안나오면 early stop
    patience = 2
    
    # epoch를 2번 돌았을 때 grad 업데이트 합니다.
    accumulation_steps = 2
    oof_pred = None

    labels = [dataset.encode_multi_class(mask, gender, age) for mask, gender, age in zip(dataset.mask_labels, dataset.gender_labels, dataset.age_labels)]
    
    # train, validation을 4:1 로 비율을 나누어 차례대로 학습을 합니다. (총 5회)
    for i, (train_idx, valid_idx) in enumerate(skf.split(dataset.image_paths, labels)):
        train_loader, val_loader = get_dataloader(dataset, train_idx, valid_idx, batch_size, num_workers)

        # -- model
        # CLASS[k] -> output size ex) mask는 3개의 output size를 가집니다 (착용, 미착용, 비정상착용)
        #target_model = model.get_model('efficientnet_b4', CLASS[k]).to(device)
        target_model = efficientnetb4(classes=CLASS[k]).to(device)

        # -- loss & matric
        loss_fn = FocalLoss()
        optimizer = AdamW(target_model.parameters(), lr = learning_rate, weight_decay=5e-4)
        scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

        best_model_path = None
        counter = 0
        best_val_acc = 0
        best_val_loss = np.inf
        
        # epoch loop
        for epoch in range(epochs):
            target_model.train()
            loss_value = 0
            matches = 0
            
            # train loop
            for idx, train_batch in enumerate(train_loader):
                inputs, labels = train_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = target_model(inputs)
                preds = torch.argmax(outputs, dim=-1)
                loss = loss_fn(outputs, labels)

                loss.backward()

                if(idx+1) % accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()

                loss_value += loss.item()
                matches += (preds == labels).sum().item()

                if(idx + 1) % train_log_interval == 0:
                    train_loss = loss_value / train_log_interval
                    train_acc = matches / batch_size / train_log_interval
                    current_lr = scheduler.get_last_lr()
                    print(
                        f"Epoch[{epoch}/{epochs}] ({idx + 1}/{len(train_loader)}) || "
                        f"training loss {train_loss:4.4} || train accuracy {train_acc:4.2%} || lr {current_lr}"
                    )
                    loss_value = 0
                    matches = 0

            scheduler.step()

            # val loop
            with torch.no_grad():
                print("Calculating validation results")
                target_model.eval()
                val_loss_items = []
                val_acc_items = []

                for val_batch in val_loader:
                    inputs, labels = val_batch
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = target_model(inputs)
                    preds = torch.argmax(outputs, dim=-1)

                    loss_item = loss_fn(outputs, labels).item()
                    acc_item = (labels == preds).sum().item()
                    val_loss_items.append(loss_item)
                    val_acc_items.append(acc_item)

                val_loss = np.sum(val_loss_items) / len(val_loader)
                val_acc = np.sum(val_acc_items) / len(valid_idx)

                # Callback: validation accuracy가 향상될수록 모델을 저장
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                if val_acc > best_val_acc: # 향상 되면 RESULT_PATH[k] 폴더에 저장
                    torch.save(target_model, os.path.join(RESULT_PATH[k], f"{i:02}_{epoch:03}_acc_{val_acc:4.2}.ckpt"))
                    best_val_acc = val_acc
                    counter = 0
                    best_model_path = os.path.join(RESULT_PATH[k], f"{i:02}_{epoch:03}_acc_{val_acc:4.2}.ckpt")

                else:
                    counter += 1

                if counter > patience:
                    print("Early Stopping")
                    print(
                        f"[Val] acc: {val_acc:4.2%}, loss: {val_loss:4.2} || "
                        f"best acc: {best_val_acc:4.2%}, best loss:{best_val_loss:4.2}"
                    )
                    break

                print(
                    f"[Val] acc: {val_acc:4.2%}, loss: {val_loss:4.2} || "
                    f"best acc: {best_val_acc:4.2%}, best loss:{best_val_loss:4.2}"
                )
    


Loaded pretrained weights for efficientnet-b4
Epoch[0/10] (20/82) || training loss 0.4045 || train accuracy 59.69% || lr [0.0001]
Epoch[0/10] (40/82) || training loss 0.1677 || train accuracy 91.56% || lr [0.0001]
Epoch[0/10] (60/82) || training loss 0.05813 || train accuracy 95.55% || lr [0.0001]
Epoch[0/10] (80/82) || training loss 0.02906 || train accuracy 97.27% || lr [0.0001]
Calculating validation results
[Val] acc: 94.35%, loss: 0.02 || best acc: 94.35%, best loss:0.02
Epoch[1/10] (20/82) || training loss 0.0209 || train accuracy 98.83% || lr [0.0001]
Epoch[1/10] (40/82) || training loss 0.0139 || train accuracy 98.67% || lr [0.0001]
Epoch[1/10] (60/82) || training loss 0.0113 || train accuracy 99.06% || lr [0.0001]
Epoch[1/10] (80/82) || training loss 0.01239 || train accuracy 99.30% || lr [0.0001]
Calculating validation results
[Val] acc: 96.16%, loss: 0.0043 || best acc: 96.16%, best loss:0.0043
Epoch[2/10] (20/82) || training loss 0.003481 || train accuracy 99.84% || lr [0.0

In [11]:
# test 데이터 경로 지정 및 로드
test_img_root = '../../input/data/eval/'
img_dir = os.path.join(test_img_root, 'images')
submission = pd.read_csv(os.path.join(test_img_root, 'info.csv'))

image_paths = [os.path.join(img_dir, img_id) for img_id in submission.ImageID]
testset = TestDataset(image_paths, resize=(256, 256))
test_loader = torch.utils.data.DataLoader(testset, shuffle=False)

In [12]:
def get_info(model):
    k, epoch, _, acc = model.split('_')
    acc1, acc2, _ = acc.split('.')
    acc = '.'.join([acc1, acc2])
    return k, acc

In [13]:
def get_best(path):
    model_list = [model for model in os.listdir(path) if not model.startswith('.')]
    best_model={}
    for model in model_list:
        k, acc = get_info(model)
        try:
            _k, _acc = get_info(best_model[int(k)])
            if float(acc) > float(_acc):
                best_model[int(k)] = model
        except:
            best_model[int(k)] = model
    
    return list(best_model.values())

In [14]:
combine = {}
n_splits = 5
# 각 도메인(mask, gender, age) 별로 루프 동작
for i, PATH in enumerate(RESULT_PATH):
    print('current working path:', PATH)
    best_models = get_best(PATH)
    oof_pred = None
    for best_model_path in best_models:
        best_model = torch.load(os.path.join(PATH, best_model_path))
        print(f"loaded best model for this fold, {best_model_path}")
        all_predictions = []
        with torch.no_grad():
            for images in test_loader:
                images = images.to(device)

                pred = best_model(images) #/ 2
                #pred += best_model(torch.flip(images, [0, 1])) / 2
                all_predictions.extend(pred.cpu().numpy())

            fold_pred = np.array(all_predictions)

        if oof_pred is None:
            oof_pred = fold_pred / n_splits
        else:
            oof_pred += fold_pred / n_splits
            
    combine[TARGET[i]] = np.argmax(oof_pred, axis=1)
        

current working path: k-fold-mask-b4
loaded best model for this fold, 02_001_acc_0.99.ckpt
loaded best model for this fold, 03_004_acc_0.99.ckpt
loaded best model for this fold, 01_003_acc_0.99.ckpt
loaded best model for this fold, 00_002_acc_0.99.ckpt
loaded best model for this fold, 04_002_acc_0.99.ckpt
current working path: k-fold-gender-b4
loaded best model for this fold, 00_001_acc_0.98.ckpt
loaded best model for this fold, 02_007_acc_0.98.ckpt
loaded best model for this fold, 03_006_acc_0.97.ckpt
loaded best model for this fold, 04_002_acc_0.98.ckpt
loaded best model for this fold, 01_005_acc_0.97.ckpt
current working path: k-fold-age-b4
loaded best model for this fold, 01_008_acc_0.86.ckpt
loaded best model for this fold, 04_005_acc_0.88.ckpt
loaded best model for this fold, 02_003_acc_0.87.ckpt
loaded best model for this fold, 00_002_acc_0.86.ckpt
loaded best model for this fold, 03_009_acc_0.87.ckpt


In [ ]:
print(combine)

In [15]:
import pickle

# save data
with open('combine.pickle','wb') as fw:
    pickle.dump(combine, fw)

# load data
# with open('combine.pickle', 'rb') as fr:
#     combine = pickle.load(fr)

In [16]:
def encode_multi_class(mask_label, gender_label, age_label) -> int:
        return mask_label * 6 + gender_label * 3 + age_label

In [17]:
multi_class =[]

In [18]:
for mask, gender, age in zip(combine['mask'], combine['gender'], combine['age']):
    multi_class.append(encode_multi_class(mask, gender, age))

In [19]:
submission = pd.read_csv(os.path.join(test_img_root, 'info.csv'))
submission['ans'] = multi_class

In [20]:
submission.to_csv('efficientb4twoAdamW.csv', index=False)